Does it get better or worse?

Presence of elements in ISO/FGDC:

- data quality
- data quality with lineage
- attribute definitions
- 

In [ ]:
%reload_ext autoreload
%autoreload 2

import json as js  # name conflict with sqla
import sqlalchemy as sqla
from sqlalchemy.orm import sessionmaker
from sqlalchemy.dialects.postgresql import *
from sqlalchemy import and_
from semproc.xml_utils import *
from mpp.models import Response
from datetime import datetime
from lxml import etree